<a href="https://colab.research.google.com/github/ADeogratias/spam_or_not_spam_model/blob/main/Spam_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# ! ls "/content/gdrive/My Drive"

In [30]:
df = pd.read_csv("/content/gdrive/My Drive/spam_or_not_spam_dataset/spam_or_not_spam.csv")

In [31]:
df.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [32]:
import re
import nltk
from keras.layers import Flatten, Dense
from keras.layers import Embedding,LSTM
from keras.models import Sequential
from keras import preprocessing
from keras.preprocessing.text import Tokenizer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
df['label'].value_counts()

0    2500
1     500
Name: label, dtype: int64

we have 500 hams and 2500 spams

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import gensim

In [36]:
X = df['email'].fillna(' ')
y = df['label']

In [37]:
# X = df['email'].values
# y = df['label'].values

In [38]:
def preprocess(text):
    text = text.str.lower()
    text = text.str.replace(r'\r', ' ')
    text = text.str.replace(r'\n', ' ')
    text = text.str.replace('[^a-zA-Z0-9]', ' ')
    return text

In [39]:
x = preprocess(X)
x = x.values
y = y.values

In [40]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2, random_state=0, shuffle = True, stratify= df['label'])
# xtrain

In [41]:
# xtest

In [42]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

def tokenizer_sequences(X):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X)
    sequences = len(tokenizer.word_index) + 1
    
    return tokenizer, sequences

In [43]:
tokenizer, sequences = tokenizer_sequences(xtrain)

In [44]:
# integer encode the documents
train_set_encoded = tokenizer.texts_to_sequences(xtrain)
test_set_encoded = tokenizer.texts_to_sequences(xtest)
print(train_set_encoded[0:2])

[[4617, 39, 21, 2032, 633, 3, 45, 11, 8, 232, 6, 667, 181, 94, 36, 4617, 718, 200, 304, 23, 410, 6, 851, 1170, 379, 13, 257, 20, 78, 4617, 1, 4617, 634, 39, 1238, 870, 46, 81, 1, 3, 76, 12, 2, 181, 2, 4617, 1171, 317, 3, 19, 3, 285, 2, 95, 45, 12, 179, 121, 302, 4617, 3, 20, 6, 466, 4303, 3, 103, 614, 12, 49, 6, 536, 41, 39, 4617, 109, 285, 13, 14, 6, 200, 18, 1073, 5, 799, 1, 1381, 4617, 2033, 1330, 58, 11943, 12, 819, 1331, 81, 70, 53, 8, 97, 7, 40, 568, 600, 34, 11, 371, 5, 436, 3, 8284, 51, 1412, 11944, 8, 102, 506, 39, 437, 2, 1469, 6509, 6510, 213, 43, 5, 133, 8285, 16016, 11, 39, 8286, 1470, 14, 5893, 339, 60, 101, 4, 492, 16, 168, 2604, 70, 13, 102, 2076, 51, 183, 10, 39, 98, 1870, 39, 71, 23, 121, 3, 45, 15, 4, 506, 81, 2, 5894, 800, 58, 18, 36, 16017, 495, 5, 5895, 2, 1526, 16018, 39, 4618, 1870, 1720, 60, 15, 22, 204, 388, 9, 275, 204, 388, 9, 16, 801, 5, 788, 9, 41, 8, 48, 6, 181, 22, 8287, 2209, 190, 8288, 25, 39, 98, 425, 69, 1871, 51, 39, 583, 3, 2, 1332, 5, 3472, 51, 38

In [45]:
import pickle
# saving
with open('vec_tokenizer.pickel', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [46]:
max_length = 6
padded_train_set = pad_sequences(train_set_encoded, maxlen=max_length, padding='post')
padded_test_set = pad_sequences(test_set_encoded, maxlen=max_length, padding='post')
# print(padded_test_set)

In [47]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip

In [48]:
# !unzip glove*.zip

In [49]:
!ls

gdrive		   glove.6B.200d.txt  glove.6B.50d.txt	sample_data
glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip	vec_tokenizer.pickel


In [50]:
# load the whole embedding into memory
embeddingz = {}
f = open('/content/glove.6B.100d.txt', encoding='utf-8')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddingz[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddingz))

Loaded 400000 word vectors.


In [51]:
#weight matrix of words in training docs
embedding_matrix = np.zeros((sequences, 100))
len(embedding_matrix)
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddingz.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector 

In [52]:
from keras.optimizers import Adam
opt = Adam(lr = 0.001)
loss = 'binary_crossentropy'

In [53]:
model = Sequential()
model.add(Embedding(sequences, 100, weights=[embedding_matrix], input_length=6))
model.add(Flatten())
model.add(Dense(10, activation= 'relu'))
model.add(Dense(1, activation= 'sigmoid' ))
# compile network
model.compile(optimizer=opt, loss = loss, metrics=['accuracy'])
# summarize defined model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 6, 100)            3032200   
_________________________________________________________________
flatten_1 (Flatten)          (None, 600)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                6010      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 3,038,221
Trainable params: 3,038,221
Non-trainable params: 0
_________________________________________________________________


In [54]:
#Fitting the data into the model
model.fit(padded_train_set, ytrain, epochs=100, verbose=1)

Epoch 1/100
75/75 [==============================] - 2s 25ms/step - loss: 0.3578 - accuracy: 0.8492
Epoch 2/100
75/75 [==============================] - 2s 24ms/step - loss: 0.2310 - accuracy: 0.9025
Epoch 3/100
75/75 [==============================] - 2s 23ms/step - loss: 0.1619 - accuracy: 0.9463
Epoch 4/100
75/75 [==============================] - 2s 24ms/step - loss: 0.1174 - accuracy: 0.9600
Epoch 5/100
75/75 [==============================] - 2s 25ms/step - loss: 0.0881 - accuracy: 0.9733
Epoch 6/100
75/75 [==============================] - 2s 24ms/step - loss: 0.0723 - accuracy: 0.9725
Epoch 7/100
75/75 [==============================] - 2s 24ms/step - loss: 0.0597 - accuracy: 0.9762
Epoch 8/100
75/75 [==============================] - 2s 24ms/step - loss: 0.0519 - accuracy: 0.9804
Epoch 9/100
75/75 [==============================] - 2s 24ms/step - loss: 0.0476 - accuracy: 0.9779
Epoch 10/100
75/75 [==============================] - 2s 24ms/step - loss: 0.0466 - accuracy: 0.9787

In [55]:
# Lets evaluate our model
loss, accuracy = model.evaluate(padded_test_set, ytest, verbose=0)
print("validation loss: ", loss, "test accuracy: ", accuracy*100,"%")

validation loss:  0.2180653065443039 test accuracy:  94.33333277702332 %


In [58]:
from tensorflow.keras.layers import Dropout

In [63]:
opt = Adam(lr = 0.0001)
loss = 'binary_crossentropy'

# Lets optimize our model
model = Sequential()
model.add(Embedding(sequences, 100, weights=[embedding_matrix], input_length=6))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(10, activation= 'relu'))
model.add(Dense(1, activation= 'sigmoid' ))
# compile network
model.compile(optimizer=opt, loss = loss, metrics=['accuracy'])
# summarize defined model
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 6, 100)            3032200   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                1010      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 11        
Total params: 3,113,621
Trainable params: 3,113,621
Non-trainable params: 0
____________________________________________

In [64]:
#Fitting the data into the model
model.fit(padded_train_set, ytrain, epochs=100, verbose=1)

Epoch 1/100
75/75 [==============================] - 2s 29ms/step - loss: 0.5668 - accuracy: 0.7563
Epoch 2/100
75/75 [==============================] - 2s 26ms/step - loss: 0.4210 - accuracy: 0.8342
Epoch 3/100
75/75 [==============================] - 2s 26ms/step - loss: 0.3950 - accuracy: 0.8333
Epoch 4/100
75/75 [==============================] - 2s 26ms/step - loss: 0.3637 - accuracy: 0.8404
Epoch 5/100
75/75 [==============================] - 2s 26ms/step - loss: 0.3185 - accuracy: 0.8575
Epoch 6/100
75/75 [==============================] - 2s 26ms/step - loss: 0.2712 - accuracy: 0.8900
Epoch 7/100
75/75 [==============================] - 2s 26ms/step - loss: 0.2374 - accuracy: 0.9150
Epoch 8/100
75/75 [==============================] - 2s 27ms/step - loss: 0.2120 - accuracy: 0.9267
Epoch 9/100
75/75 [==============================] - 2s 27ms/step - loss: 0.2001 - accuracy: 0.9283
Epoch 10/100
75/75 [==============================] - 2s 27ms/step - loss: 0.1796 - accuracy: 0.9379

In [65]:
# Lets evaluate our model
loss, accuracy = model.evaluate(padded_test_set, ytest, verbose=0)
print("validation loss: ", loss, "test accuracy: ", accuracy*100,"%")

validation loss:  0.2626582980155945 test accuracy:  94.49999928474426 %


In [66]:
# save model
model.save('spam_model.h5')

In [67]:
# load model
from keras.models import load_model
model = load_model( 'spam_model.h5' )

In [68]:
# downloadin the model and use it in Heroku
from google.colab import files
files.download('spam_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>